In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
칼럼 =['연번','자치구','담당','구분','집행일시','시분값', \
       '집행장소','집행주소','결재방법','집행금액','집행목적/집행내역/내용','대상인원','50만원이상 결재시 증빙서류','비고']
기본경로 = r'C:\Users\user\1코드들\2018년_정공센_알권리감시단_업무추진비_코드\Files\공개자료_업데이트\\'
파일위치경로 = r'C:/Users/user/Desktop/구청별 업무추진비(엑셀)/'
구청목록 = [ 
    '종로구','중구','용산구','성동구','광진구','동대문구','중랑구','성북구','강북구','도봉구','노원구',
    '은평구','서대문구','마포구','양천구','강서구','구로구','금천구','영등포구','동작구','관악구','서초구',
    '강남구','송파구','강동구','서울특별시'
            ]   #26개
요일 = {'Monday':'월', 'Tuesday':'화', 'Wednesday':'수', 'Thursday':'목', 'Friday':'금', 'Saturday':'토', 'Sunday':'일'}
일정시점_이후 = '2014-07-01'

## 날짜정제

In [3]:
import re

In [8]:
구청별=dict()

for 구청이름 in 구청목록:
    try:
        구청별[구청이름]=pd.read_excel(파일위치경로+'/원본/'+구청이름+'.xlsx',index_col='연번')
    except FileNotFoundError:
        print('파일이 없습니다:'+구청이름)
    except :
        print(구청이름)

파일이 없습니다:서울특별시


In [9]:
def change_date_format(x):
    try:
        if len(x)>=10 or x=='-':
            return x.replace('.','-')
        if len(x)==8:
            imp_str = x[0:5]+'0'+x[5:7]+'0'+x[7]
            return imp_str.replace('.','-')
        앞 =re.compile('20[0-9][0-9].[1-9].[0-9][0-9]')
        if 앞.findall(x):
            imp_str =앞.findall(x)[0]
            imp_str = imp_str[0:5]+'0'+imp_str[5:]
            return imp_str.replace('.','-')
        else:
            뒤 =re.compile('20[0-9][0-9].[0-9][1-9].[1-9]')
            imp_str = 뒤.findall(x)[0]
            imp_str = imp_str[0:8]+'0'+imp_str[8]
            return imp_srt.replace('.','-')
    except:
        print(x)

In [10]:
for 구청이름 in 구청목록:
    try:
        구청별[구청이름].집행일시 = 구청별[구청이름].집행일시.apply(lambda x: change_date_format(str(x)))
        구청별[구청이름].시분값 = 구청별[구청이름].시분값.astype(str)
        구청별[구청이름] = 구청별[구청이름][구청별[구청이름]['집행일시']>=일정시점_이후]
        #구청별[구청이름].집행일시 = pd.to_datetime(구청별[구청이름].집행일시)
    except KeyError:
        print('파일이 없습니다:'+구청이름)
    except :
        print(구청별[구청이름].집행일시)

파일이 없습니다:서울특별시


## 요일입력

In [11]:
import datetime
요일사전 = {0:'월', 1:'화', 2:'수', 3:'목', 4:'금', 5:'토', 6:'일'}

In [12]:
구청별['종로구'].head()

,자치구,담당,구분,집행일시,시분값,집행장소,집행주소,결재방법,집행금액,집행목적/집행내역/내용,대상인원,50만원이상 결재시 증빙서류,비고
연번,,,,,,,,,,,,,
89,종로구,이상수,의장,2014-07-10 00:00:00,20:12:11,정일품,서울 종로구 종로5길 58,카드,105000,의정활동 관련 간담회,-,-,-
90,종로구,이상수,의장,2014-07-10 00:00:00,12:46:34,밥줘,서울 종로구 종로3길 30,카드,55000,의정활동 관련 간담회,-,-,-
91,종로구,이상수,의장,2014-07-11 00:00:00,13:02:48,밥줘,서울 종로구 종로3길 30,카드,132000,의정활동 관련 간담회,-,-,-
92,종로구,이상수,의장,2014-07-12 00:00:00,20:57:51,백제고기집,서울 종로구 종로35길,카드,130000,의정활동 관련 간담회,-,-,-
93,종로구,이상수,의장,2014-07-14 00:00:00,13:57:05,고향보리밥,서울 종로구 삼청동,카드,58000,의정활동 관련 간담회,-,-,-


In [13]:
for 구청이름 in 구청목록:
    try:
        임시_구청 = 구청별[구청이름]
    except KeyError:
        print("목록에 없습니다:"+구청이름)
        continue
    try:
        임시_구청_집행일시 = pd.to_datetime(임시_구청.집행일시)
    except:    
        print("집행일시를 변환하지 못합니다."+구청이름)
        임시_구청_집행일시 = pd.to_datetime(임시_구청[임시_구청.집행일시 != '-'].집행일시)
        임시_구청['요일'] = 임시_구청_집행일시.apply(lambda x : 요일사전[x.weekday()])
        임시_구청.요일 = 임시_구청.요일.fillna('-')
        구청별[구청이름] = 임시_구청
        구청별[구청이름].to_excel(파일위치경로+구청이름+'(정제).xlsx')
        continue
    임시_구청['요일'] = 임시_구청_집행일시.apply(lambda x : 요일사전[x.weekday()])
    구청별[구청이름] = 임시_구청.reset_index(drop=True)
    구청별[구청이름].to_excel(파일위치경로+구청이름+'(정제).xlsx')

목록에 없습니다:서울특별시


## 중복 확인

In [14]:
중복확인= dict()
for 구청이름, 임시_구청 in 구청별.items():
    중복확인[구청이름] = 임시_구청[임시_구청.duplicated()]

In [15]:
for i,j in 중복확인.items():
    if len(j)!=0:
        print(i, len(j))

종로구 6
용산구 25
노원구 3
양천구 4
강서구 1
금천구 103
동작구 3
관악구 14
강남구 2
강동구 1


In [16]:
중복확인['양천구']

,자치구,담당,구분,집행일시,시분값,집행장소,집행주소,결재방법,집행금액,집행목적/집행내역/내용,대상인원,50만원이상 결재시 증빙서류,비고,요일
2423,양천구,배여운,복지건설위원장,2015-11-04,12:46:19,우리네코다리,-,-,60000,-,5,-,-,수
2427,양천구,배여운,복지건설위원장,2015-11-11,21:17:14,북해도,-,-,156000,-,9,-,-,수
2431,양천구,배여운,복지건설위원장,2015-11-12,13:32:06,목동마차,-,-,51000,-,5,-,-,목
2432,양천구,배여운,복지건설위원장,2015-11-12,13:32:41,목동마차,-,-,27000,-,3,-,-,목
